# Convolutional Deep Belief Network

## Import



In [ ]:
def get_cdbn_library():
  files = ["CDBN.py", "CRBM.py", "DATA_HANDLER.py"]
  repository_url = "https://raw.githubusercontent.com/SimoDR/Convolutional_Deep_Belief_Network/master/src"
  for file in files:
    ! wget -O {file} {repository_url}{file}

get_cdbn_library()

In [ ]:
import tensorflow.compat.v1 as tf
import tensorflow_probability as tfp
from tensorflow.keras.utils import to_categorical
tf.disable_eager_execution()
# tf.disable_v2_behavior()
import random

import CRBM as crbm
import CDBN as cdbn
import DATA_HANDLER as data_handler

## Data

I load the MNIST dataset and perform some preprocessing

In [ ]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels_normal),(test_images, test_labels_normal) = mnist.load_data()

# train_images = train_images[:10000]
# train_labels_normal = train_labels_normal[:10000]

# one hot encoding
train_labels = to_categorical(train_labels_normal,10)
test_labels = to_categorical(test_labels_normal,10)

# scale the pixels values in [0,1]
train_images = train_images/255
test_images = test_images/255

train_images = train_images.reshape(60000,784)
test_images = test_images.reshape(10000,784)

### Semi-supervised learning

I train the CDBN on the whole mnist dataset but without labels: only for the last softmax layers I use a subset of the dataset with labels.

In [ ]:
# I use the whole dataset to train the CDBN layers
unsupervised_training_images = train_images

# I pick a subset of the whole dataset for supervised training
# on the last softmax layer
sampler = random.sample(range(60000), 1000)
supervised_training_images = train_images[sampler]
supervised_training_labels = train_labels[sampler]

I use the data handler to import data in the model

In [ ]:
mnist_dataset = data_handler.DATA_HANDLER(unsupervised_training_images, 
                                          supervised_training_images, 
                                          supervised_training_labels, 
                                          test_images, 
                                          test_labels)
#mnist_dataset.do_whiten()
sess = tf.Session()

## Model



In [ ]:
my_cdbn = cdbn.CDBN('mnist_cdbn', 20, '/home/', mnist_dataset, sess, verbosity = 2)

my_cdbn.add_layer('layer_1', fully_connected = False, v_height = 28, v_width = 28, v_channels = 1, f_height = 11, f_width = 11, f_number = 40, 
               init_biases_H = -3, init_biases_V = 0.01, init_weight_stddev = 0.01, 
               gaussian_unit = True, gaussian_variance = 0.2, 
               prob_maxpooling = True, padding = True, 
               learning_rate = 0.0001, learning_rate_decay = 0.6, momentum = 0.5, decay_step = 50000,  
               weight_decay = 2.0, sparsity_target = 0.003, sparsity_coef = 0.1)

my_cdbn.add_layer('layer_2', fully_connected = False, v_height = 14, v_width = 14, v_channels = 40, f_height = 7, f_width = 7, f_number = 40, 
               init_biases_H = -3, init_biases_V = 0.025, init_weight_stddev = 0.025, 
               gaussian_unit = False, gaussian_variance = 0.2, 
               prob_maxpooling = True, padding = True, 
               learning_rate = 0.005, learning_rate_decay = 0.8, momentum = 0.9, decay_step = 50000,  
               weight_decay = 0.1, sparsity_target = 0.1, sparsity_coef = 0.1)

# my_cdbn.add_layer('layer_3', fully_connected = True, v_height = 1, v_width = 1, v_channels = 40*7*7, f_height = 1, f_width = 1, f_number = 200, 
#                init_biases_H = -3, init_biases_V = 0.025, init_weight_stddev = 0.025, 
#                gaussian_unit = False, gaussian_variance = 0.2, 
#                prob_maxpooling = False, padding = False, 
#                learning_rate = 0.0025, learning_rate_decay = 0.5, momentum = 0.9, decay_step = 50000,  
#                weight_decay = 0.1, sparsity_target = 0.1, sparsity_coef = 0.1)

my_cdbn.add_softmax_layer(10, 0.10)

my_cdbn.lock_cdbn()

tf.get_variable_scope().reuse_variables()


## Training

In [ ]:
# tf.debugging.enable_check_numerics() # to debug nan's

my_cdbn.manage_layers(['layer_1','layer_2'],[],[5000,10000], [1,1], 10000, restore_softmax = False, fine_tune = True)

## Evaluation

In [ ]:
my_cdbn.do_eval()